########## Step 1: One Hot encoding our categorical data ########## 
#Goal: pre-processing data for feature selection with RFE ( Recursive Feature Elimination)

In [2]:
import numpy as np
import pandas as pd

# import preprocessing from sklearn
from sklearn import preprocessing

In [3]:
# load dataset
pathDataset = "../Dataset/results.xlsx"
original_dataset = pd.read_excel(pathDataset)

#Format date column to date time
#dataset['date'] = pd.to_datetime(X.date)
original_dataset['date'] =  pd.to_datetime(original_dataset['date'], format='%Y/%m/%d')
original_dataset['date'] =  pd.to_datetime(original_dataset['date'], format='%Y%-m-%d')
#dataset['date'] =pd.to_datetime(dataset['date'], format='%Y/%m/%d', errors='ignore')
#dataset['date'] =pd.to_datetime(dataset['date'], format='%Y%-m-%d', errors='ignore')

#X['date'] = X['date'].dt.strftime('%Y/%m/%d')
original_dataset.dtypes

#Display 3 first rows
original_dataset.head(140)


,date,home_team,away_team,home_score,away_score,tournament,city,country
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland
1,1872-11-30,England,Scotland,4,2,Friendly,London,England
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland
3,1875-03-06,England,Scotland,2,2,Friendly,London,England
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland
5,1876-03-25,Scotland,Wales,4,0,Friendly,Glasgow,Scotland
6,1877-03-03,England,Scotland,1,3,Friendly,London,England
7,1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales
8,1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland
9,1878-03-23,Scotland,Wales,9,0,Friendly,Glasgow,Scotland


In [4]:
# limit to categorical data using df.select_dtypes()
columns_categorical_subset = original_dataset.select_dtypes(include=[object])
columns_categorical_subset.head(3)

,home_team,away_team,tournament,city,country
0,Scotland,England,Friendly,Glasgow,Scotland
1,England,Scotland,Friendly,London,England
2,Scotland,England,Friendly,Glasgow,Scotland


In [5]:
# check original shape
columns_categorical_subset.shape

(38685, 5)

In [6]:
# view columns using df.columns
columns_categorical_subset.columns

Index(['home_team', 'away_team', 'tournament', 'city', 'country'], dtype='object')

In [7]:
# TODO: create a LabelEncoder object and fit it to each feature in X


# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()
le

# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
final_dataset = original_dataset.apply(le.fit_transform)
final_dataset.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country
0,0,184,64,0,0,47,559,201
1,0,65,180,4,2,47,890,68
2,1,184,64,2,1,47,559,201
3,2,65,180,2,2,47,890,68
4,3,184,64,3,0,47,559,201


In [8]:
# TODO: create a OneHotEncoder object, and fit it to all of X

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(final_dataset)

# 3. Transform
onehotlabels = enc.transform(final_dataset).toarray()
onehotlabels.shape
# Explication: 
# as you can see, you've the same number of rows 891
# but now you've so many more columns due to how we changed all the categorical data into numerical data

(38685, 17429)

In [9]:
onehotlabels

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [10]:
type(onehotlabels)

numpy.ndarray

# Step 2: Selection Featrue with RFE Algorith

In [11]:
#final_dataset = pd.DataFrame(new_dataset)
final_dataset.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country
0,0,184,64,0,0,47,559,201
1,0,65,180,4,2,47,890,68
2,1,184,64,2,1,47,559,201
3,2,65,180,2,2,47,890,68
4,3,184,64,3,0,47,559,201


In [12]:
final_dataset_values = final_dataset.values
X = final_dataset_values[:,0:7]
Y = final_dataset_values[:,7]

In [13]:
Y

array([201,  68, 201, ..., 236,  10, 184])

# RFE Algorithm

In [ ]:
# Import packages
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# feature extraction
model = LogisticRegression()
rfe = RFE(model, 4)
fit = rfe.fit(X, Y)
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

# Principal Component Analyse Algorithm

In [ ]:
#Import Package
from sklearn.decomposition import PCA

# feature extraction
pca = PCA(n_components=3)
fit = pca.fit(X)
# summarize components
print("Explained Variance: %s" % fit.explained_variance_ratio_)
print(fit.components_)

# Feature Importance Algorithm

In [ ]:
# feature extraction
model = ExtraTreesClassifier()
model.fit(X, Y)
print(model.feature_importances_)